## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps
import json
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Georgetown,MY,5.4112,100.3354,84.13,80,20,5.01,few clouds
1,1,Avarua,CK,-21.2078,-159.7750,82.45,74,40,12.66,scattered clouds
2,2,Vila Velha,BR,-20.3297,-40.2925,76.95,86,0,8.05,clear sky
3,3,Ushuaia,AR,-54.8000,-68.3000,49.66,57,20,16.11,few clouds
4,4,Kaitangata,NZ,-46.2817,169.8464,54.93,57,45,8.43,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Georgetown,MY,5.4112,100.3354,84.13,80,20,5.01,few clouds
1,1,Avarua,CK,-21.2078,-159.7750,82.45,74,40,12.66,scattered clouds
2,2,Vila Velha,BR,-20.3297,-40.2925,76.95,86,0,8.05,clear sky
9,9,Atuona,PF,-9.8000,-139.0333,78.69,78,38,20.87,light rain
11,11,New Norfolk,AU,-42.7826,147.0587,73.71,37,0,1.99,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

Unnamed: 0     0
City           0
Country        1
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna(how='any')

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Georgetown,MY,84.13,few clouds,5.4112,100.3354,
1,Avarua,CK,82.45,scattered clouds,-21.2078,-159.7750,
2,Vila Velha,BR,76.95,clear sky,-20.3297,-40.2925,
9,Atuona,PF,78.69,light rain,-9.8000,-139.0333,
11,New Norfolk,AU,73.71,clear sky,-42.7826,147.0587,
15,Kapaa,US,79.12,clear sky,22.0752,-159.3190,
18,Puerto Ayora,EC,75.54,scattered clouds,-0.7393,-90.3518,
19,Faanui,PF,79.81,moderate rain,-16.4833,-151.7500,
21,Tulagi,SB,84.00,broken clouds,-9.0996,160.1476,
22,Saint-Philippe,RE,78.39,light rain,-21.3585,55.7679,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
   
    #print(json.dumps(hotels, indent=4, sort_keys=True))    
    try:
        hotel_df.loc[index,'Hotel Name'] = hotels['results'][0]['name']
        
    except:
        print('Hotel not found...skipping')
        
print("------DONE------")

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
------DONE------


In [9]:
hotel_df.loc[hotel_df['Hotel Name'] == '']

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
113,Wattegama,LK,72.52,overcast clouds,6.7989,81.4808,
131,Yaan,NG,75.47,clear sky,7.3833,8.5667,
178,Airai,TL,73.54,broken clouds,-8.9266,125.4092,
264,Poum,NC,79.54,light rain,-20.2333,164.0167,
365,Beloha,MG,74.12,light rain,-25.1667,45.0500,
412,Thinadhoo,MV,81.82,few clouds,0.5333,72.9333,
527,Madimba,TZ,75.40,light rain,-10.4000,40.3333,


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_data = hotel_df.copy()
hotel_data["Hotel Name"].replace('',np.nan,inplace=True)
hotel_data.dropna(subset=["Hotel Name"], inplace=True)
hotel_data.isnull().sum()

City           0
Country        0
Max Temp       0
Description    0
Lat            0
Lng            0
Hotel Name     0
dtype: int64

In [11]:
# 8a. Create the output File (CSV)
output_data_file = '../Vacation_Search/WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
hotel_data.to_csv(output_data_file, index_label="City_ID")

In [12]:
hotel_data.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Georgetown,MY,84.13,few clouds,5.4112,100.3354,Cititel Penang
1,Avarua,CK,82.45,scattered clouds,-21.2078,-159.7750,Paradise Inn
2,Vila Velha,BR,76.95,clear sky,-20.3297,-40.2925,Quality Suites Vila Velha
9,Atuona,PF,78.69,light rain,-9.8000,-139.0333,Villa Enata
11,New Norfolk,AU,73.71,clear sky,-42.7826,147.0587,The Shingles Riverside Cottages


In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Description} and {Max Temp} °F </dd>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_data.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_data[["Lat", "Lng"]]
hotel_info


['\n<dl>\n<dt>Hotel Name</dt><dd>Cititel Penang</dd>\n<dt>City</dt><dd>Georgetown</dd>\n<dt>Country</dt><dd>MY</dd>\n<dt>Weather Description</dt><dd>few clouds and 84.13 °F </dd>\n\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Paradise Inn</dd>\n<dt>City</dt><dd>Avarua</dd>\n<dt>Country</dt><dd>CK</dd>\n<dt>Weather Description</dt><dd>scattered clouds and 82.45 °F </dd>\n\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Quality Suites Vila Velha</dd>\n<dt>City</dt><dd>Vila Velha</dd>\n<dt>Country</dt><dd>BR</dd>\n<dt>Weather Description</dt><dd>clear sky and 76.95 °F </dd>\n\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Villa Enata</dd>\n<dt>City</dt><dd>Atuona</dd>\n<dt>Country</dt><dd>PF</dd>\n<dt>Weather Description</dt><dd>light rain and 78.69 °F </dd>\n\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>The Shingles Riverside Cottages</dd>\n<dt>City</dt><dd>New Norfolk</dd>\n<dt>Country</dt><dd>AU</dd>\n<dt>Weather Description</dt><dd>clear sky and 73.71 °F </dd>\n\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Sheraton Kauai Coconut

In [14]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_data["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))